In [13]:
import pandas as pd
import pickle
import numpy as np

### Export

In [3]:
wmt17_seg = pickle.load(open('../../wmt16-19-metrics-shared-task/data/pickles/wmt17-seg_level-agg.pkl', 'rb'))
df = wmt17_seg[wmt17_seg.lp.str.endswith('en')].copy()
df.head(1)

,lp,system,sid,output,source,reference,score,sentBLEU
560,cs-en,online-A.0,33,"However, conventional trains consisting of wag...",Klasické soupravy složené z vagonů a tažené lo...,Conventional trains consisting of carriages an...,0.492423,0.191983


In [4]:
r = open('./references.txt', 'wt')
o = open('./outputs.txt', 'wt')
[ r.write(i) for i in df.reference.tolist() ]
[ o.write(i) for i in df.output.tolist() ]
r.close()
o.close()

In [11]:
workers = [i.split('\t')[0] for i in open('../exp_3/workers.tsv', 'rt') ]
r = open('./references_30_annotators.txt', 'wt')
o = open('./outputs_30_annotators.txt', 'wt')


for i, j in zip(df.reference.tolist(), df.output.tolist()):
    for _, worker in zip(range(0, 30), workers):
        r.write('%s %s' % (worker, i))
        o.write(j)

r.close()
o.close()

### Load

In [5]:
df['score:wmt16_seg_score'] = [ float(i.strip()) for i in open('./wmt16_seg_score', 'rt') ]
df['score:wmt16_sys_score_nonorm'] = [ float(i.strip()) for i in open('./wmt16_sys_score_nonorm', 'rt') ]
df['score:wmt16_sys_score_norm'] = [ float(i.strip()) for i in open('./wmt16_sys_norm_score', 'rt') ]
df['score:wmt16_sys_score_annotators'] = [ float(i.strip()) for i in open('./wmt16_sys_annotators_score', 'rt') ]

In [16]:
annotators_30 = [ float(i.strip()) for i in open('./wmt16_sys_annotators_score_30', 'rt') ]

average_over = 5
lam
averaged_score = [ np.mean(annotators_30[i:i+average_over]) for i in range(0, len(annotators_30), 30)  ]
df['score:wmt16_sys_score_annotators_5']  = averaged_score

In [26]:
df['score:wmt16_sys_score_nonorm_postnorm'] = (df['score:wmt16_sys_score_nonorm'] - df['score:wmt16_sys_score_nonorm'].mean()) / df['score:wmt16_sys_score_nonorm'].std()

### Evaluate

In [27]:
num_metrics = sum('score' in i for i in df.columns)
df[['lp'] + [i for i in df.columns if 'score' in i]].groupby(['lp']).corr()[0::num_metrics].T

lp,cs-en,de-en,fi-en,lv-en,ru-en,tr-en,zh-en
,score,score,score,score,score,score,score
score,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
score:wmt16_seg_score,0.725746,0.746315,0.866960,0.811805,0.797504,0.797673,0.774295
score:wmt16_sys_score_nonorm,0.668307,0.682114,0.822099,0.784904,0.699685,0.756208,0.725322
score:wmt16_sys_score_norm,0.748234,0.781193,0.884388,0.832010,0.818631,0.833564,0.804027
score:wmt16_sys_score_annotators,0.741701,0.772890,0.879062,0.828708,0.813443,0.818809,0.784629
score:wmt16_sys_score_annotators_5,0.749547,0.775762,0.879770,0.830140,0.813326,0.820042,0.784548
score:wmt16_sys_score_annotators_10,0.749881,0.776138,0.879805,0.829955,0.813707,0.819852,0.784461
score:wmt16_sys_score_annotators_15,0.749388,0.776399,0.879598,0.829959,0.813858,0.819509,0.784389
score:wmt16_sys_score_annotators_30,0.749527,0.776495,0.879632,0.829954,0.813958,0.819550,0.784247
